# Fine tuning SMLs con Hugging Face

## Loging

Para poder subir el resultado del entrenamiento al hub debemos logearnos primero, para ello necesitamos un token

Para crear un token hay que ir a la página de [setings/tokens](https://huggingface.co/settings/tokens) de nuestra cuenta, nos aparecerá algo así

![User-Access-Token-dark](http://maximofn.com/wp-content/uploads/2024/03/User-Access-Token-dark.png)

Le damos a `New token` y nos aparecerá una ventana para crear un nuevo token

![new-token-dark](http://maximofn.com/wp-content/uploads/2024/03/new-token-dark.png)

Le damos un nombre al token y lo creamos con el rol `write`, o con el rol `Fine-grained`, que nos permite seleccionar exactamente qué permisos tendrá el token

Una vez creado lo copiamos y lo pegamos a continuación

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

## Dataset

Ahora nos descargamos un dataset, en este caso nos vamos a descargar uno de reviews de [Yelp](https://www.yelp.com) que es una plataforma de reviews de todo tipo de negocios

In [1]:
from datasets import load_dataset

dataset = load_dataset("mteb/amazon_reviews_multi", "en")

Vamos a verlo un poco

In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 5000
    })
})

Vemos que tiene un conjunto de entrenamiento con 200.000 muestras, uno de validación con 5.000 muestras y uno de test de 5.000 muestras

Vamos a ver un ejemplo del conjunto de entrenamiento

In [3]:
from random import randint

idx = randint(0, len(dataset['train']) - 1)
dataset['train'][idx]

{'id': 'en_0907914',
 'text': 'Mixed with fir it’s passable\n\nNot the scent I had hoped for . Love the scent of cedar, but this one missed',
 'label': 3,
 'label_text': '3'}

Vemos que tiene la review en el campo `text` y la puntuación que le ha dado el usuario en el campo `label`

Como vamos a hacer un modelo de clasificación de textos, necesitamos saber cuantas clases vamos a tener

In [4]:
num_classes = len(dataset['train'].unique('label'))
num_classes

5

Vamos a tener 5 clases, ahora vamos a ver el valor mínimo de estas clases para saber si la puntuación comienza en 0 o en 1. Para ello usamos el método `unique`

In [5]:
dataset.unique('label')

{'train': [0, 1, 2, 3, 4],
 'validation': [0, 1, 2, 3, 4],
 'test': [0, 1, 2, 3, 4]}

El mínimo valor va a ser 0

Para entrenar, las etiquetas tienen que estar en un campo llamado `labels`, mientras que en nuestro dataset está en un campo que se llama `label`, por lo que creamos el nuevo campo `lables` con el mismo valor que `label`

Creamos una función que haga lo que queremos

In [6]:
def set_labels(example):
    example['labels'] = example['label']
    return example

Aplicamos la función al dataset

In [7]:
dataset = dataset.map(set_labels)

Vamos a ver cómo queda el dataset

In [8]:
dataset['train'][idx]

{'id': 'en_0907914',
 'text': 'Mixed with fir it’s passable\n\nNot the scent I had hoped for . Love the scent of cedar, but this one missed',
 'label': 3,
 'label_text': '3',
 'labels': 3}

## Tokenizador

Como en el dataset tenemos las reviews en texto, necesitamos tokenizarlas para poder meter los tokens al modelo

In [9]:
from transformers import AutoTokenizer

checkpoint = "openai-community/gpt2"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Ahora creamos una función para tokenizar el texto. Lo vamos a hacer de manera que todas las sentencias tenan la mismas longitud, de manera que el tokenizador truncará cuando sea necesario y añadirá tokens de padding cuando sea necesario. Además le indicamos que devuelva tensores de pytorch

Hacemos que la longitud de cada sentencia sea de 768 tokens porque estamos usando el modelo pequeño de GPT2, que como vimos en el post de [GPT2](https://maximofn.com/gpt2/#Arquitectura) tiene una dimensión de embedding de 768 tokens

In [10]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=768, return_tensors="pt")

Vamos a probar a tokenizar un texto

In [11]:
tokens = tokenize_function(dataset['train'][idx])

ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

Nos da un error porque el tokenizador de GPT2 no tiene un token para padding y nos pide que asignemos uno, además nos sugiere hacer `tokenizer.pad_token = tokenizer.eos_token`, así que lo hacemos

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

Volvemos a probar la función de tokenización

In [ ]:
tokens = tokenize_function(dataset['train'][idx])
tokens['input_ids'].shape, tokens['attention_mask'].shape

(torch.Size([1, 768]), torch.Size([1, 768]))

Ahora que hemos comprobado que la función tokeniza bien, aplicamos esta función al dataset, pero además la aplicamos por batches para que se ejecute más rápido

Además aprovechamos y eliminamos las columnas que no vamos a necesitar

In [ ]:
dataset = dataset.map(tokenize_function, batched=True, remove_columns=['text', 'label', 'id', 'label_text'])

Vemos ahora cómo queda el dataset

In [12]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'label_text', 'labels'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['id', 'text', 'label', 'label_text', 'labels'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'label_text', 'labels'],
        num_rows: 5000
    })
})

Vemos que tenemos los campos 'labels', 'input_ids' y 'attention_mask', que es lo que nos interesa para entrenar

## Modelo

Instanciamos un modelo para clasificación de secuencias y le indicamos el número de clases que tenemos

In [13]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=num_classes)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at openai-community/gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Nos dice que los pesos de la capa `score` han sido inicializados de manera aleatoria y que tenemos que reentrenarlos, vamos a ver por qué pasa esto

El modelo de GPT2 sería este

In [14]:
from transformers import AutoModelForCausalLM

casual_model = AutoModelForCausalLM.from_pretrained(checkpoint)

Mientras que el modelo de GPT2 para generar texto es este

Vamos a ver su arquitectura

In [15]:
casual_model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

Y ahora la arquitectura del modelo que vamos a usar para clasificar las reviews

In [16]:
model

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=5, bias=False)
)

De aquí hay dos cosas que mencionar

 + La primera es que en ambos, la primera capa tiene dimensiones de 50257x768, que corresponde a 50257 posibles tokens del vocabulario de GPT2 y a 768 dimensiones del embedding, por lo que hemos hecho bien en tokenizar las reviews con un tamaño de 768 tokens
 + La segunda es que el modelo `casual` (el de generación de texto) tiene al final una capa `Linear` que genera 50257 valores, es decir, es la encargada de predecir el siguiente token y a posible token le da un valor. Mientras que el modelo de clasificación tiene una capa `Linear` que solo genera 5 valores, uno por cada clase, lo que nos dará la probabilidad de que la review pertenezca a cada clase

Por eso nos salía el mensaje de que los pesos de la capa `score` habían sido inicializados de manera aleatoria, porque la librería transformers ha eliminado la capa `Linear` de 768x50257 y ha añadido una capa `Linear` de 768x5, la ha inicializado con valores aleatorios y nosotros la tenemos que entrenar para nuestro problema en particular

Borramos el modelo casual, porque no lo vamos a usar

In [17]:
del casual_model

## Trainer

Vamos ahora a configurar los argumentos del entrenamiento

In [18]:
from transformers import TrainingArguments

metric_name = "accuracy"
model_name = "GPT2-small-finetuned-amazon-reviews-en-classification"
LR = 2e-5
BS_TRAIN = 28
BS_EVAL = 40
EPOCHS = 3
WEIGHT_DECAY = 0.01

training_args = TrainingArguments(
    model_name,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=LR,
    per_device_train_batch_size=BS_TRAIN,
    per_device_eval_batch_size=BS_EVAL,
    num_train_epochs=EPOCHS,
    weight_decay=WEIGHT_DECAY,
    lr_scheduler_type="cosine",
    warmup_ratio = 0.1,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
)

Definimos una métrica para el dataloader de validación

In [19]:
import numpy as np
from evaluate import load

metric = load("accuracy")

def compute_metrics(eval_pred):
    print(eval_pred)
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

Definimos ahora el trainer

In [20]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Entrenamos

In [21]:
trainer.train()

  0%|          | 0/600000 [00:00<?, ?it/s]

ValueError: You should supply an encoding or a list of encodings to this method that includes input_ids, but you provided ['label', 'labels']

Nos vuelve a salir un error porque el modelo no tiene asignado un token de padding, así que al igual que con el tokenizador se lo asignamos

In [25]:
model.config.pad_token_id = model.config.eos_token_id

Volvemos a crear los argumentos del trainer con el nuevo modelo, que ahora si tiene token de padding, el trainer y volvemos a entrenar

In [ ]:
training_args = TrainingArguments(
    model_name,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=LR,
    per_device_train_batch_size=BS_TRAIN,
    per_device_eval_batch_size=BS_EVAL,
    num_train_epochs=EPOCHS,
    weight_decay=WEIGHT_DECAY,
    lr_scheduler_type="cosine",
    warmup_ratio = 0.1,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
    logging_dir="./runs",
)

trainer = Trainer(
    model,
    training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Ahora que hemos visto que está todo bien podemos entrenar

In [28]:
trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy
1,0.807400,0.820341,0.652000
2,0.751900,0.802189,0.654600
3,0.718100,0.810221,0.657800


TrainOutput(global_step=21429, training_loss=0.7846888848762739, metrics={'train_runtime': 26367.7801, 'train_samples_per_second': 22.755, 'train_steps_per_second': 0.813, 'total_flos': 2.35173445632e+17, 'train_loss': 0.7846888848762739, 'epoch': 3.0})

## Evaluación

Una vez entrenado evaluamos sobre el dataset de test

In [29]:
trainer.evaluate(eval_dataset=dataset['test'])

{'eval_loss': 0.7973636984825134,
 'eval_accuracy': 0.6626,
 'eval_runtime': 76.3016,
 'eval_samples_per_second': 65.529,
 'eval_steps_per_second': 1.638,
 'epoch': 3.0}

## Publicar el modelo

Ya tenemos nuestro modelo entrenado, ya podemos compartirlo con el mundo, así que primero creamos una model card

In [ ]:
trainer.create_model_card()

Y ya lo podemos publicar. Como lo primero que hemos hecho ha sido loguearnos con el hub de huggingface, lo podremos subir a nuestro hub sin ningún problema

In [ ]:
trainer.push_to_hub()